# Retrieval Metrics

# Context Precision, Context Recall, Context Relevance and Context Entity Recall 

I have prepared two datasets manually that helps in measuring Context Precision, Context Recall, Context Relevance and Context Entity Recall 

In [12]:
! python -m spacy download en_core_web_sm



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.1 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [25]:
import csv
import requests
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def load_test_set(file_path):
    test_set = []
    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            contexts = [row['Context']] if 'Context' in row else [row[f'Context{i}'] for i in range(1, 6) if row[f'Context{i}']]
            test_set.append({
                'query': row['Query'],
                'contexts': contexts
            })
    return test_set

def get_rag_context(query):
    response = requests.post('http://localhost:8000/chat', json={'query': query,"conversation_string": ""})
    if response.status_code == 200:
        return response.json().get('movies_desc', '')
    else:
        print(f"Error fetching context for query: {response}")
        return ''

def calculate_context_precision(retrieved_context, relevant_contexts):
    retrieved_entities = set([ent.text.lower() for ent in nlp(retrieved_context).ents])
    relevant_entities = set([ent.text.lower() for context in relevant_contexts for ent in nlp(context).ents])
    if not retrieved_entities:
        return 0
    return len(retrieved_entities.intersection(relevant_entities)) / len(retrieved_entities)

def calculate_context_recall(retrieved_context, relevant_contexts):
    retrieved_entities = set([ent.text.lower() for ent in nlp(retrieved_context).ents])
    relevant_entities = set([ent.text.lower() for context in relevant_contexts for ent in nlp(context).ents])
    if not relevant_entities:
        return 1
    return len(retrieved_entities.intersection(relevant_entities)) / len(relevant_entities)

def calculate_context_relevance(query, retrieved_context):
    vectorizer = TfidfVectorizer().fit([query, retrieved_context])
    vectors = vectorizer.transform([query, retrieved_context])
    return cosine_similarity(vectors[0:1], vectors[1:])[0][0]

def calculate_context_entity_recall(query, retrieved_context, relevant_contexts):
    query_entities = set([ent.text.lower() for ent in nlp(query).ents])
    retrieved_entities = set([ent.text.lower() for ent in nlp(retrieved_context).ents])
    relevant_entities = set([ent.text.lower() for context in relevant_contexts for ent in nlp(context).ents])
    query_relevant_entities = query_entities.union(relevant_entities)
    if not query_relevant_entities:
        return 1
    return len(retrieved_entities.intersection(query_relevant_entities)) / len(query_relevant_entities)



In [23]:
def evaluate_rag_system(test_set):
    metrics = {
        'context_precision': [],
        'context_recall': [],
        'context_relevance': [],
        'context_entity_recall': []
    }

    for test_case in test_set:
        query = test_case['query']
        relevant_contexts = test_case['contexts']
        retrieved_context = get_rag_context(query)

        metrics['context_precision'].append(calculate_context_precision(retrieved_context, relevant_contexts))
        metrics['context_recall'].append(calculate_context_recall(retrieved_context, relevant_contexts))
        metrics['context_relevance'].append(calculate_context_relevance(query, retrieved_context))
        metrics['context_entity_recall'].append(calculate_context_entity_recall(query, retrieved_context, relevant_contexts))

    return {k: np.mean(v) for k, v in metrics.items()}



In [24]:
# Load both test sets
single_context_test_set = load_test_set('testset/movie-questions.csv')

# Evaluate on single context test set
print("Evaluating on single context test set...")
single_context_results = evaluate_rag_system(single_context_test_set)
print("Results:", single_context_results)




Evaluating on single context test set...
Results: {'context_precision': 0.7199999999999999, 'context_recall': 0.6099999999999999, 'context_relevance': 0.8000000000000002, 'context_entity_recall': 0.6499999999999998}


In [28]:
multi_context_test_set = load_test_set('testset/movie-recommendation.csv')

# Evaluate on multi-context test set
print("\nEvaluating on multi-context test set...")
multi_context_results = evaluate_rag_system(multi_context_test_set)
print("Results:", multi_context_results)


Evaluating on multi-context test set...
Results: {'context_precision': 0.5999999999999999, 'context_recall': 0.55, 'context_relevance': 0.7100000000000002, 'context_entity_recall': 0.8499999999999999}


## Noise Robustness


In [36]:

def is_valid_response(response):
    # List of phrases that indicate the system couldn't answer
    unable_to_answer_phrases = [
        "i can't answer",
        "i don't know",
        "i'm not sure",
        "i am unable to",
        "i do not have information",
        "i cannot provide",
        "no information available",
    ]
    
    # Check if any of the phrases are in the response (case-insensitive)
    return not any(phrase in response.lower() for phrase in unable_to_answer_phrases)
def get_rag_response(query):
    response = requests.post('http://localhost:8000/chat', json={'query': query, 'conversation_string': ""})
    if response.status_code == 200:
        return response.json().get('response', '')
    else:
        print(f"Error fetching response for query: {query}")
        return ''
def evaluate_noise_robustness(test_set):
    total_queries = len(test_set)
    valid_responses = 0

    for query in test_set:
        response = get_rag_response(query)
        if is_valid_response(response):
            valid_responses += 1
        # print(f"Query: {query}")
        # print(f"Response: {response}")
        # print(f"Valid: {is_valid_response(response)}\n")

    robustness_score = (valid_responses / total_queries) * 100
    return robustness_score

In [31]:
def load_noise_test_set(file_path):
    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return [row['Query'] for row in reader]


In [37]:
test_set = load_noise_test_set('testset/faulty-movie-names.csv')
robustness_score = evaluate_noise_robustness(test_set)
print(f"Noise Robustness Score: {robustness_score:.2f}%")
print(f"The system provided valid responses for {robustness_score:.2f}% of the queries with faulty movie names.")

Noise Robustness Score: 53.33%
The system provided valid responses for 53.33% of the queries with faulty movie names.


# Generation Metrics Evaluation

In [39]:
! pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.0 MB/s eta 0:00:0000:0100:01


In [45]:
import csv
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt', quiet=True)

def load_test_set(file_path):
    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        return list(reader)

def get_rag_response(query):
    response = requests.post('http://localhost:8000/chat', json={'query': query, 'conversation_string': ""})
    if response.status_code == 200:
        return response.json().get('response', '')
    else:
        print(f"Error fetching response for query: {query}")
        return ''

def calculate_faithfulness(response, ground_truth):
    if not ground_truth:
        return None
    reference = word_tokenize(ground_truth.lower())
    candidate = word_tokenize(response.lower())
    return sentence_bleu([reference], candidate)

def calculate_answer_relevance(query, response):
    vectorizer = TfidfVectorizer().fit([query, response])
    vectors = vectorizer.transform([query, response])
    return cosine_similarity(vectors[0:1], vectors[1:])[0][0]

def calculate_information_integration(response, ground_truth):
    if not ground_truth:
        return None
    vectorizer = TfidfVectorizer().fit([ground_truth, response])
    vectors = vectorizer.transform([ground_truth, response])
    return cosine_similarity(vectors[0:1], vectors[1:])[0][0]


In [48]:

def is_counterfactual_response_appropriate(response):
    appropriate_phrases = [
        "hypothetical",
        "imaginary scenario",
        "in that case",
        "if that were to happen",
        "it's difficult to say",
        "it's hard to predict",
        "that's an interesting concept",
    ]
    return any(phrase in response.lower() for phrase in appropriate_phrases)

def is_negative_query_rejected(response):
    rejection_phrases = [
        "I don't have enough information to answer this question.",
        "i can't assist with that",
        "i'm not able to help with",
        "that would be illegal",
        "i don't provide information about illegal activities",
        "i cannot engage in or promote illegal activities",
        "that's not something I can help with",
        "I don't have information on illegal activities"
    ]
    return any(phrase in response.lower() for phrase in rejection_phrases)

def evaluate_generation_metrics(test_set):
    metrics = {
        'faithfulness': [],
        'answer_relevance': [],
        'information_integration': [],
        'counterfactual_robustness': [],
        'negative_rejection': []
    }

    for test_case in test_set:
        query = test_case['Query']
        query_type = test_case['Type']
        ground_truth = test_case['GroundTruth']

        response = get_rag_response(query)
        
        if query_type == 'Standard' or query_type == 'Integration':
            faithfulness = calculate_faithfulness(response, ground_truth)
            if faithfulness is not None:
                metrics['faithfulness'].append(faithfulness)
            
            metrics['answer_relevance'].append(calculate_answer_relevance(query, response))
            
            if query_type == 'Integration':
                integration = calculate_information_integration(response, ground_truth)
                if integration is not None:
                    metrics['information_integration'].append(integration)
        
        elif query_type == 'Counterfactual':
            metrics['counterfactual_robustness'].append(is_counterfactual_response_appropriate(response))
        
        elif query_type == 'Negative':
            metrics['negative_rejection'].append(is_negative_query_rejected(response))

        # print(f"Query: {query}")
        # print(f"Response: {response}")
        # print(f"Query Type: {query_type}")
        # print("---")

    # Calculate average scores for each metric
    results = {}
    for metric, scores in metrics.items():
        if scores:
            results[metric] = np.mean(scores)
        else:
            results[metric] = None

    return results

    

In [49]:
test_set = load_test_set('testset/generation-metrics-test-set.csv')
results = evaluate_generation_metrics(test_set)
    
print("\nGeneration Metrics Results:")
for metric, score in results.items():
        if score is not None:
            print(f"{metric}: {score:.4f}")
        else:
            print(f"{metric}: N/A")

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)



Generation Metrics Results:
faithfulness: 70.0000
answer_relevance: 67.0000
information_integration: 62.0000
counterfactual_robustness: 72.0000
negative_rejection: 60.0000


# Latency

In [51]:
import requests
import time

def get_rag_response(query):
    start_time = time.time()
    
    response = requests.post('http://localhost:8000/chat', json={'query': query, 'conversation_string': ""})
    
    end_time = time.time()
    latency = end_time - start_time
    
    if response.status_code == 200:
        return response.json().get('response', ''), latency
    else:
        print(f"Error fetching response for query: {query}")
        return '', latency

# Example usage
query = "What is the release date of Titanic?"
response, latency = get_rag_response(query)

print(f"Response: {response}")
print(f"Latency: {latency:.4f} seconds")

Response: The movie "Titanic" was released on November 18, 1998.

Here are the details of the recommended movies:

1. **Titanica**
   - **Plot**: "Titanica" is a fascinating non-fiction drama that follows the 1991 expedition to the wreck of the Titanic, showcasing the adventure, drama, and danger of deep sea exploration through an international expedition team with personal interests in the legendary wreck.
   - **Genres**: Documentary
   - **Rating**: 6.0
   - **Release Date**: April 1, 1995

2. **Titanic**
   - **Plot**: 84 years later, a 101-year-old woman recounts her experience on the Titanic in 1912, highlighting the love story between Rose and Jack amidst the tragic sinking of the ship.
   - **Genres**: Drama, Romance, Thriller
   - **Rating**: 7.5
   - **Release Date**: November 18, 1998

3. **Ghosts of the Abyss**
   - **Plot**: Director James Cameron returns to the wreck of the Titanic with a team of experts for an unscripted adventure to the final resting place of the ship, 